In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import os
import time
import datetime
import numpy as np
from numpy import zeros
from numpy import ones
from cvae import CVAE
import random
from tensorboard.plugins.hparams import api as hp
from sklearn.decomposition import PCA

tf.keras.backend.set_floatx('float64')
tf.config.experimental_run_functions_eagerly(True)

random.seed(0)
HP_CLUSTER = hp.HParam('cluster', hp.Discrete([0,1,2,3]))
HP_LATENT_DIM = hp.HParam('latent_dim',    hp.Discrete([3,4,5,10,20]))
#HP_LEARNING_RATE = hp.HParam('learning rate', hp.Discrete([0.01,0.001]))
#HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu','linear', 'sigmoid'])) 
HP_UNITS = hp.HParam('units', hp.Discrete([10, 15, 20, 25]))
#HP_LAYERS = hp.HParam('layers', hp.Discrete([1, 2, 3]))
#HP_NORM = hp.HParam('norm', hp.Discrete(['L1', 'L2']))
HP_MOVING_KL_COEFF = hp.HParam('moving_kl', hp.Discrete(['constant']))
#HP_MOVING_KL_COEFF = hp.HParam('moving_kl', hp.Discrete(['linear', 'sigmoid', 'constant']))
HP_KL_COEFF = hp.HParam('kl_coeff', hp.Discrete([10]))
#HP_KL_COEFF = hp.HParam('kl_coeff', hp.Discrete([0.01, 0.1, 0.5, 1, 5, 10, 100]))

MSE = 'mse'
KL = 'kl'
LOSS = 'loss'
MSE_test = 'mse_test'
KL_test = 'kl_test'
LOSS_test = 'loss_test'

with tf.summary.create_file_writer('logs/latent_dim_layers').as_default():
    hp.hparams_config(
        hparams=[HP_CLUSTER, HP_LATENT_DIM, HP_UNITS, HP_MOVING_KL_COEFF, HP_KL_COEFF], 
        #hparams=[HP_CLUSTER, HP_LATENT_DIM,HP_LEARNING_RATE, HP_ACTIVATION, HP_UNITS,
        #         HP_LAYERS, HP_NORM, HP_MOVING_KL_COEFF, HP_KL_COEFF],
        metrics=[hp.Metric(MSE, display_name='Mean Square Error'),
                 hp.Metric(KL, display_name='Kullbach Leibler divergence'),
                 hp.Metric(LOSS, display_name='Loss'),
                 hp.Metric(MSE_test, display_name='Mean Square Error Test'),
                 hp.Metric(KL_test, display_name='Kullbach Leibler divergence Test'),
                 hp.Metric(LOSS_test, display_name='Loss Test')],
    )

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
activation = 'relu'
clustering_name = 'nmf'
kl_coeff = 10
moving_kl_coeff = 'constant'

df_train = pd.read_feather('../data/train_' + clustering_name + '.feather')
df_test = pd.read_feather('../data/test_' + clustering_name + '.feather')
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])
clusters = pd.read_feather('../data/sum_up_' + clustering_name + '.feather')
nb_clust = clusters.cluster.size
clusters = clusters.set_index('cluster')

epochs = 50
input_dim = 48
nb_dates_for_train = len(df_train['date'].unique())
nb_dates_for_test = len(df_test['date'].unique())  # - nb_dates_for_train

nb_layers_encoder = 2
nb_layers_decoder = 2
weights_decoder = None
weights_encoder = None

cols_clust = []
for c in clusters.index:
    df_train['cluster_{:02}'.format(c)] = np.array(1 * (df_train['cluster'] == c), dtype='float64')
    df_test['cluster_{:02}'.format(c)] = np.array(1 * (df_test['cluster'] == c), dtype='float64')
    cols_clust.append('cluster_{:02}'.format(c))

cols_data = []
cols_price = []
cols_temp = ['temp_smooth_n']

for i in range(48):
    cols_data.append('consumption_{:02}'.format(i))
    cols_price.append('low_{:02}'.format(i))
    cols_price.append('high_{:02}'.format(i))

for i in range(3):
    cols_temp.append('pca_temp_{:1}'.format(i))
cols_cond = cols_temp + ['pos_y', 'work_day'] + cols_price 
cond_dim = len(cols_cond) 

initializer = tf.keras.initializers.glorot_uniform(seed=0)


In [ ]:
def train_model(hparams):
    
    units_layers_encoder = [units, 2*latent_dim]
    units_layers_decoder = [units, input_dim]
    
    train_mse = tf.keras.metrics.Mean('train_mse', dtype=tf.float64)
    train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float64)
    train_kl = tf.keras.metrics.Mean('train_kl', dtype=tf.float64)
    test_mse = tf.keras.metrics.Mean('test_mse', dtype=tf.float64)
    test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float64)
    test_kl = tf.keras.metrics.Mean('test_kl', dtype=tf.float64)

    df_train_c = df_train.loc[df_train['cluster_{:02}'.format(c)] == 1]
    df_test_c = df_test.loc[df_test['cluster_{:02}'.format(c)] == 1]

    conso_max = max(df_train_c.loc[:, cols_data].max())
    conso_min = min(df_train_c.loc[:, cols_data].min())
    df_train_c.loc[:, cols_data] = (df_train_c.loc[:, cols_data] - conso_min) / (conso_max - conso_min)
    df_test_c.loc[:, cols_data] = (df_test_c.loc[:, cols_data] - conso_min) / (conso_max - conso_min)

    d0 = df_train_c.index[0]
    df_train_tf = tf.reshape(df_train_c.loc[d0, cols_data], shape=[1, 48])
    cond_train_tf = tf.reshape(df_train_c.loc[d0, cols_cond], shape=[1, cond_dim])
    for d in df_train_c.index[1:len(df_train_c.index)]:
        df_train_tf = tf.concat([df_train_tf, tf.reshape(df_train_c.loc[d, cols_data], shape=[1, 48])], 0)
        cond_train_tf = tf.concat([cond_train_tf, tf.reshape(df_train_c.loc[d, cols_cond], shape=[1, cond_dim])], 0)

    d0 = df_test_c.index[0]
    df_test_tf = tf.reshape(df_test_c.loc[d0, cols_data], shape=[1, 48])
    cond_test_tf = tf.reshape(df_test_c.loc[d0, cols_cond], shape=[1, cond_dim])
    for d in df_test_c.index[1:len(df_test_c.index)]:
        df_test_tf = tf.concat([df_test_tf, tf.reshape(df_test_c.loc[d, cols_data], shape=[1, 48])], 0)
        cond_test_tf = tf.concat([cond_test_tf, tf.reshape(df_test_c.loc[d, cols_cond], shape=[1, cond_dim])], 0)

    name = 'cluster' + str(c) + '_' + 'latentdim_' + str(latent_dim) + '_units_' + str(units) 
        
    model = CVAE(input_dim, latent_dim, cond_dim,
                 full_covariance_matrix=False,
                 nb_layers_encoder=nb_layers_encoder,
                 units_layers_encoder=units_layers_encoder,
                 nb_layers_decoder=nb_layers_decoder,
                 units_layers_decoder=units_layers_decoder,
                 initializer_encoder=initializer,
                 weights_encoder=weights_encoder,
                 initializer_decoder=initializer,
                 weights_decoder=weights_decoder,
                 activation=activation,
                 optimizer=optimizer,
                 loss='L2')

    for epoch in range(epochs):
        if moving_kl == 'linear':
            eta = (epoch / epochs) * kl_coeff
        if moving_kl == 'sigmoid':
            x = -6 + 12 * epoch / epochs
            eta = (1 / (1 + np.math.exp(-x))) * kl_coeff
        if moving_kl == 'constant':
            eta = 1 * kl_coeff
        model.train_step(df_train_tf, cond_train_tf, eta)

    for d in range(df_train_tf.shape[0]):
        x = tf.reshape(df_train_tf[d,].numpy(), shape=[1, model.input_dim])
        cond = tf.reshape(cond_train_tf[d,].numpy(), shape=[1, model.cond_dim])
        train_mse(model.compute_mse(x, cond))
        train_loss(model.compute_loss(x, cond))
        train_kl(model.compute_kl_err(x, cond))

    for d_t in range(df_test_tf.shape[0]):
        x = tf.reshape(df_test_tf[d_t,].numpy(), shape=[1, input_dim])
        cond = tf.reshape(cond_test_tf[d_t,].numpy(), shape=[1, cond_dim])
        test_mse(model.compute_mse(x, cond))
        test_loss(model.compute_loss(x, cond))
        test_kl(model.compute_kl_err(x, cond))

    #model.decoder_net.save('../data/decoders/hyp_opt/' + name + '.h5')

    return train_mse.result(), train_loss.result(), train_kl.result(), test_mse.result(), test_loss.result(), test_kl.result()


def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        train_mse, train_loss, train_kl, test_mse, test_loss, test_kl = train_model(hparams)
        tf.summary.scalar(MSE, train_mse, step=1)
        tf.summary.scalar(LOSS, train_loss, step=1)
        tf.summary.scalar(KL, train_kl, step=1)
        tf.summary.scalar(MSE_test, test_mse, step=1)
        tf.summary.scalar(LOSS_test, test_loss, step=1)
        tf.summary.scalar(KL_test, test_kl, step=1)

In [ ]:
session_num = 0
for c in HP_CLUSTER.domain.values:
    for latent_dim in HP_LATENT_DIM.domain.values:
        for moving_kl in HP_MOVING_KL_COEFF.domain.values:
            for kl_coeff in HP_KL_COEFF.domain.values:
    #            for learning_rate in HP_LEARNING_RATE.domain.values:
    #                for activation in HP_ACTIVATION.domain.values:
    #                    for norm in HP_NORM.domain.values:
    #                        for layers in HP_LAYERS.domain.values:
    #                             if layers >1:
                for units in HP_UNITS.domain.values:
                    hparams = {HP_CLUSTER: c,
                        HP_LATENT_DIM: latent_dim,
   #                    HP_LEARNING_RATE: learning_rate,
   #                    HP_ACTIVATION: activation,
                        HP_UNITS: units,
   #                    HP_LAYERS: layers,
                        HP_MOVING_KL_COEFF: moving_kl,
                        HP_KL_COEFF: kl_coeff,
   #                    HP_NORM: norm,
                        }
    #                             else:
    #                                hparams = {HP_CLUSTER: c,
    #                                    HP_LATENT_DIM: latent_dim,
    #                                    HP_LEARNING_RATE: learning_rate,
    #                                    HP_ACTIVATION: activation,
    #                                    HP_UNITS: 0,
    #                                    HP_LAYERS: layers,
    #                                    HP_MOVING_KL_COEFF: moving_kl,
    #                                    HP_KL_COEFF: kl_coeff,
    #                                    HP_NORM: norm,
    #                                    }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hp_architecture/' + run_name, hparams)
            session_num += 1
